<a href="https://colab.research.google.com/github/tstreet24/Street_Portfolio/blob/main/SlotMarketSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras_tuner
!pip install arch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import numpy as np
import tensorflow as tf
from tensorflow import keras
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from arch import arch_model
import matplotlib.pyplot as plt
import pickle
import os
tf.random.set_seed(42)

# **INITIAL CODE TO GET PAST TWO YEARS OF DATA**

In [ ]:
# Function to fetch tickers
def fetch_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    sp500_table = tables[0]
    sp500_tickers = sp500_table['Symbol'].tolist()
    return sp500_tickers

sp500_tickers = fetch_sp500_tickers()
print(sp500_tickers)


In [ ]:
# Pulling closing prices from beginning of 2022 to february 2024
sp500_tickers = fetch_sp500_tickers()

end_date = datetime.today()
start_date = datetime(2022, 1, 1)

all_closing_prices = pd.DataFrame()

for ticker in sp500_tickers:
    stock = yf.Ticker(ticker)
    hist = stock.history(start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    all_closing_prices[ticker] = hist['Close']

print(all_closing_prices)


In [ ]:
# Dropping a column that was NA
all_closing_prices = all_closing_prices.dropna(1)

In [ ]:
all_closing_prices.to_csv("closing_prices.csv")

# **Data Preparation**

In [ ]:
# Read data
df = pd.read_csv("/content/drive/My Drive/HODL_Project/closing_prices.csv")
df.head(5)

In [ ]:
# Normalize (min-max)
df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce') # don't apply to 0th column, which is date
normalized_df = df.copy()
normalized_df.iloc[:, 1:] = (df.iloc[:, 1:] - df.iloc[:, 1:].min()) / (df.iloc[:, 1:].max() - df.iloc[:, 1:].min())
normalized_df.head(5)

In [ ]:
# Filter data to 2023 points
normalized_df['Date'] = pd.to_datetime(normalized_df['Date'], utc = True)
df_2023 = normalized_df[(normalized_df['Date'].dt.year == 2023) | (normalized_df['Date'].dt.year == 2022)]
df_2024_first_entry = normalized_df[normalized_df['Date'].dt.year == 2024].head(1)
df_initial = pd.concat([df_2023, df_2024_first_entry])
df_initial.tail(5)

# Important Functions

In [ ]:
# Function to properly shape X and y for LSTM
def create_sequences(series, lag):
    inputs = []
    targets = []
    for i in range(len(series) - lag):
        inputs.append(series[i:(i + lag)])
        targets.append(series[i + lag])
    return np.array(inputs), np.array(targets)

In [ ]:
# Function to perform walk forward validation with varying parameters
def walk_forward_validation(series, lag, units, dropout, val_steps, epochs):
    series = np.array(series) if not isinstance(series, np.ndarray) else series

    X, y = create_sequences(series, lag)
    X = X.reshape((X.shape[0], X.shape[1], 1))

    split_idx = len(X) - val_steps

    mses = []

    # Define architecture
    input_layer = keras.Input(shape=(lag, 1))
    lstm_layer = keras.layers.LSTM(units, activation='relu')(input_layer)
    dropout_layer = keras.layers.Dropout(dropout)(lstm_layer)
    output_layer = keras.layers.Dense(1)(dropout_layer)
    model = keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])

    # Validation loop
    for step in range(val_steps - 1):
        X_train, y_train = X[:split_idx + step], y[:split_idx + step]
        X_val, y_val = X[split_idx + step:split_idx + step + 1], y[split_idx + step:split_idx + step + 1]


        model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=0)

        mse = model.evaluate(X_val, y_val, verbose=0)[1]  # [1] for mse
        mses.append(mse)

    # Return the MSEs for each validation step and the final model
    return mses, model


# Model Building and Evaluation, ONE STOCK for EXPLORATION

In [ ]:
series = df_initial.iloc[:,1]
lag = 10
X, y = create_sequences(series, lag)
X = X.reshape((X.shape[0], X.shape[1], 1))

In [ ]:
X_train = X[:-1]
X_test = X[-1:]
y_train = y[:-1]
y_test = y[-1:]

In [ ]:
input = keras.Input(shape=(lag, 1))
lstm = keras.layers.LSTM(30, activation='relu')(input)
dropout = keras.layers.Dropout(0.2)(lstm)
output = keras.layers.Dense(1)(dropout)
model = keras.Model(inputs=input, outputs=output)




model.compile(optimizer='adam', loss='mse', metrics = ['mse'])

In [ ]:
model.fit(X_train,
          y_train,
          epochs=20,
          batch_size=32)

In [ ]:
model.evaluate(X_test, y_test)

# EXPERIMENT - Determining adequate number of epochs

In [ ]:
# Function to perform epoch experiment with walk forward validation
def epoch_experiment(df, lag, units, dropout, val_steps, epoch_values):

    # Dictionary with results
    results = {column: {} for column in df.columns}

    for column in df.columns:
        print(f"Processing column: {column}")
        series = df[column].values

        for epochs in epoch_values:
            mses, _ = walk_forward_validation(series, lag, units, dropout, val_steps, epochs)
            avg_mse = np.mean(mses)
            results[column][epochs] = avg_mse

    # Plotting
    for column in df.columns:
        plt.figure(figsize=(10, 6))
        epochs_list = list(results[column].keys())
        avg_mses = list(results[column].values())

        model_params_label = f'Walk Forward Validation MSE - Lag: {lag}, Units: {units}, Dropout: {dropout}'

        plt.plot(epochs_list, avg_mses, marker='o', label=model_params_label)

        plt.title(f'Walk Forward Validation MSE by Epoch for {column}')
        plt.xlabel('Epochs')
        plt.ylabel('Average MSE')
        plt.xticks(epochs_list)
        plt.grid(False)

        plt.legend()
        plt.show()

In [ ]:
# Parameters for experiment
lag = 10
units = 30
dropout = 0.2
val_steps = 8
epoch_range = range(1,21)

# Randomly pick columns (TSLA, UNH, ETR)
np.random.seed(15773)
random_num = np.random.choice(range(1, df_initial.shape[1] + 1), 3, replace=False)
df = df_initial.iloc[:,[random_num[0]
                        ,random_num[1]
                        ,random_num[2]
                        ]]

# Call function
epoch_experiment(df, lag, units, dropout, val_steps, epoch_range)

### Conclusion: Around 5 epochs seems reasonable

# EXPERIMENT - Determine lag length

In [ ]:
# Function to perform lag experiment using walk forward validation
def lag_experiment(df, lag_values, units, dropout, val_steps, constant_epochs):
    # List to store all results
    results = []

    for column in df.columns:
        print(f"Processing column: {column}")
        series = df[column].values

        for lag in lag_values:
            mses, _ = walk_forward_validation(series, lag, units, dropout, val_steps, constant_epochs)
            avg_mse = np.mean(mses)

            results.append({
                'Column': column,
                'Lag': lag,
                'Average_MSE': avg_mse
            })

    results_df = pd.DataFrame(results)

    return results_df

In [ ]:
# Parameters for experiment
lag_values = [7, 14, 28]
units = 30
dropout = 0.2
val_steps = 8
epochs = 5

# Randomly pick columns
np.random.seed(15773)
random_num = np.random.choice(range(1, df_initial.shape[1] + 1), 3, replace=False)
df = df_initial.iloc[:,[random_num[0]
                        ,random_num[1]
                        ,random_num[2]
                        ]]

# Call function
results = lag_experiment(df, lag_values, units, dropout, val_steps, epochs)
lag_agg = results.groupby('Lag')['Average_MSE'].mean().reset_index()
lag_agg

### CONCLUSION: We will use a lag of 7 herein

# EXPERIMENT - Parameter tuning with keras.tuner, no walk forward validation, for warm start

In [ ]:
# 0 - TSLA
series = df.iloc[:,0]
lag = 7
X, y = create_sequences(series, lag)
X = X.reshape((X.shape[0], X.shape[1], 1))

X_train = X[:-1]
X_test = X[-1:]
y_train = y[:-1]
y_test = y[-1:]


def build_model(hp):
    input_layer = keras.Input(shape=(lag, 1))
    lstm_layer = keras.layers.LSTM(
        hp.Int('lstm_units', min_value=10, max_value=100, step=10), activation='relu')(input_layer)
    dropout_layer = keras.layers.Dropout(
        hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.05))(lstm_layer)
    output_layer = keras.layers.Dense(1)(dropout_layer)
    model = keras.Model(inputs=input_layer, outputs=output_layer)

    model.compile(optimizer='adam', loss='mse')
    return model

tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    project_name='tuner0'
)

tuner.search(X_train, y_train, epochs=5, validation_data = (X_test, y_test))

In [ ]:
# 0 - TSLA
best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hp.values

In [ ]:
# 1 - UNH
series = df.iloc[:,1]
lag = 7
X, y = create_sequences(series, lag)
X = X.reshape((X.shape[0], X.shape[1], 1))

X_train = X[:-1]
X_test = X[-1:]
y_train = y[:-1]
y_test = y[-1:]


def build_model(hp):
    input_layer = keras.Input(shape=(lag, 1))
    lstm_layer = keras.layers.LSTM(
        hp.Int('lstm_units', min_value=10, max_value=100, step=10), activation='relu')(input_layer)
    dropout_layer = keras.layers.Dropout(
        hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.05))(lstm_layer)
    output_layer = keras.layers.Dense(1)(dropout_layer)
    model = keras.Model(inputs=input_layer, outputs=output_layer)

    model.compile(optimizer='adam', loss='mse')
    return model

tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    project_name='tuner1'
)

tuner.search(X_train, y_train, epochs=5, validation_data = (X_test, y_test))

In [ ]:
# 1 - UNH
best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hp.values

In [ ]:
# 2 - ETR
series = df.iloc[:,2]
lag = 7
X, y = create_sequences(series, lag)
X = X.reshape((X.shape[0], X.shape[1], 1))

X_train = X[:-1]
X_test = X[-1:]
y_train = y[:-1]
y_test = y[-1:]


def build_model(hp):
    input_layer = keras.Input(shape=(lag, 1))
    lstm_layer = keras.layers.LSTM(
        hp.Int('lstm_units', min_value=10, max_value=100, step=10), activation='relu')(input_layer)
    dropout_layer = keras.layers.Dropout(
        hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.05))(lstm_layer)
    output_layer = keras.layers.Dense(1)(dropout_layer)
    model = keras.Model(inputs=input_layer, outputs=output_layer)

    model.compile(optimizer='adam', loss='mse')
    return model

tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    project_name='tuner2'
)

tuner.search(X_train, y_train, epochs=5, validation_data = (X_test, y_test))

In [ ]:
# 2 - ETR
best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hp.values

### CONCLUSION - for stock 0, 90 units and 10% dropout   ;    for stock 1, 100 units and 0% dropout    ;    for stock 2, 50 units and 15% dropout

# EXPERIMENT - Manual parameter tuning for units and dropout rate using warm starts from previous experiment

In [ ]:
# Function to perform manual parameter tuning
def parameter_tuning(df, lag, unit_range, dropout_range, val_steps, epochs):
    # Initialize a list to store the results
    all_results = []

    for column in df.columns:
        series = df[column].values

        for units in unit_range:

            for dropout in dropout_range:
                mses, _ = walk_forward_validation(series, lag, units, dropout, val_steps, epochs)
                avg_mse = np.mean(mses)

                all_results.append({
                    'Column': column,
                    'Units': units,
                    'Dropout': dropout,
                    'Average_MSE': avg_mse
                })

    results_df = pd.DataFrame(all_results)

    return results_df


In [ ]:
# Parameters for experiment
lag = 7
unit_range = [80, 90, 100, 110]
dropout_range = [0, 0.05, 0.1, 0.15]
val_steps = 8
epochs = 5

# Randomly pick columns
np.random.seed(15773)
random_num = np.random.choice(range(1, df_initial.shape[1] + 1), 3, replace=False)
df = df_initial.iloc[:,[random_num[0]
                        ,random_num[1]
                        ,random_num[2]
                        ]]

# Call function
results = parameter_tuning(df, lag, unit_range, dropout_range, val_steps, epochs)
lag_agg = results.groupby(['Units', 'Dropout'])['Average_MSE'].mean().reset_index()
lag_agg.rename(columns={'Average_MSE': 'Aggregated_Average_MSE'}, inplace=True)

In [ ]:
lag_agg

### CONCLUSION - 90 units, 15% dropout rate

# EXPERIMENT - Stacking LSTMS

In [ ]:
# Function to create architecture based on number of stacks
def build_stacks(lag, units, dropout, num_stacks):
    input_layer = keras.Input(shape=(lag, 1))
    x = input_layer

    # Adjust units based on the number of stacks
    adjusted_units = int(units / num_stacks)

    for _ in range(num_stacks):
        x = keras.layers.LSTM(adjusted_units, activation='relu', return_sequences=(_ < num_stacks - 1))(x)
        x = keras.layers.Dropout(dropout)(x)

    output_layer = keras.layers.Dense(1)(x)
    model = keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')

    return model


In [ ]:
# Function to perform stacks experiment using walk forward validation
def evaluate_stacked_models(df, lag, units, dropout, val_steps, epochs, stack_options):
    results = []

    for column in df.columns:
        series = df[column].values

        for num_stacks in stack_options:
            print(f"Evaluating: {column} with {num_stacks} Stacks")
            model = build_stacks(lag, units, dropout, num_stacks)

            mses, _ = walk_forward_validation(series, lag, units, dropout, val_steps, epochs)
            avg_mse = np.mean(mses)

            results.append({
                'Column': column,
                'Num_Stacks': num_stacks,
                'Average_MSE': avg_mse
            })

    return pd.DataFrame(results)


In [ ]:
# Parameters for experiment
lag = 7
units = 90
dropout = 0.15
val_steps = 8
epochs = 5
stack_options = range(1, 11)

# Randomly pick columns
np.random.seed(15773)
random_num = np.random.choice(range(1, df_initial.shape[1] + 1), 3, replace=False)
df = df_initial.iloc[:,[random_num[0]
                        ,random_num[1]
                        ,random_num[2]
                        ]]

# Call function
results = evaluate_stacked_models(df, lag, units, dropout, val_steps, epochs, stack_options)
lag_agg = results.groupby(['Num_Stacks'])['Average_MSE'].mean().reset_index()
lag_agg.rename(columns={'Average_MSE': 'Aggregated_Average_MSE'}, inplace=True)

In [ ]:
lag_agg

### Conclusion: 6 stacks are best

# Creating models

In [ ]:
base_save_path = '/content/drive/My Drive/HODL_Project/Models/'

for i in range(498, 498):
  series = df_initial.iloc[:,i]
  lag = 7
  X, y = create_sequences(series, lag)
  X = X.reshape((X.shape[0], X.shape[1], 1))

  X_train = X[:-1]
  X_test = X[-1:]
  y_train = y[:-1]
  y_test = y[-1:]

  input = keras.Input(shape=(lag, 1))
  lstm_1 = keras.layers.LSTM(15, activation='relu', return_sequences=True)(input)
  dropout_1 = keras.layers.Dropout(0.15)(lstm_1)
  lstm_2 = keras.layers.LSTM(15, activation='relu', return_sequences=True)(dropout_1)
  dropout_2 = keras.layers.Dropout(0.15)(lstm_2)
  lstm_3 = keras.layers.LSTM(15, activation='relu', return_sequences=True)(dropout_2)
  dropout_3 = keras.layers.Dropout(0.15)(lstm_3)
  lstm_4 = keras.layers.LSTM(15, activation='relu', return_sequences=True)(dropout_3)
  dropout_4 = keras.layers.Dropout(0.15)(lstm_4)
  lstm_5 = keras.layers.LSTM(15, activation='relu', return_sequences=True)(dropout_4)
  dropout_5 = keras.layers.Dropout(0.15)(lstm_5)
  lstm_6 = keras.layers.LSTM(15, activation='relu')(dropout_5)
  dropout_6 = keras.layers.Dropout(0.15)(lstm_6)
  output = keras.layers.Dense(1)(dropout_6)
  model = keras.Model(inputs=input, outputs=output)

  model.compile(optimizer='adam', loss='mse', metrics = ['mse'])


  model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0, validation_data=(X_test, y_test))

  model_save_path = os.path.join(base_save_path, f'model_column_{i}.h5')
  model.save(model_save_path)
  print(f'Model saved to {model_save_path}')



# GARCH MODELS

In [ ]:
df

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], utc = True)
df_2023 = df[(df['Date'].dt.year == 2023) | (df['Date'].dt.year == 2022)]
df_2024_first_entry = df[df['Date'].dt.year == 2024].head(1)
df = pd.concat([df_2023, df_2024_first_entry])

In [ ]:
log_returns = np.log(df.iloc[:, 1:] / df.iloc[:, 1:].shift(1))
log_returns = log_returns.dropna()
df = log_returns

In [ ]:
df

In [ ]:
base_save_path = '/content/drive/My Drive/HODL_Project/GARCH_Models/'
for i in range(0,df.shape[1]):  # Start=1 for naming files starting from 1
    # Select the series excluding the last row
    series = df.iloc[:,i]

    # Fit a GARCH(1,1) model
    model = arch_model(series, vol='Garch', p=1, q=1, mean='constant', dist='Normal')
    results = model.fit(disp='off')

    # Construct the save path for the model
    model_save_path = os.path.join(base_save_path, f'garch_model_{i+1}.pkl')

    # Save the model using pickle
    with open(model_save_path, 'wb') as f:
        pickle.dump(results, f)

    print(f"Model saved to {model_save_path}")

# LSTM Evaluation

## Functions

In [ ]:
def update_model(model, train_series, test_series):
  norm_preds = []
  mse = []

  train_series = np.array(train_series)
  test_series = np.array(test_series)

  size_test = len(test_series)

  for i in range(1,size_test):

      #Prep Data
      model_series = np.append(train_series, test_series[0:i+1])

      #Prep data for model
      X, y = create_sequences(model_series, 7)
      X = X.reshape((X.shape[0], X.shape[1], 1))
      X_train = X[:-1]
      X_test = X[-1:]
      y_train = y[:-1]
      y_test = y[-1:]

      #Update model
      updated_model = model.fit(X_train, y_train, epochs=5, batch_size=32, verbose = 0)
      # Predict the stock price for the next day
      pred_norm_price = model.predict(X_test)[0][0]
      #mse
      y_test = y_test[0]
      error = np.square(y_test - pred_norm_price)

      # Store the prediction & error
      norm_preds.append(pred_norm_price)
      mse.append(error)

      #Save Model if last run
      if i == size_test - 1:
        model_filename = f'/content/drive/My Drive/HODL_Project/Updated_LSTM/model_column_{j}.h5'
        model.save(model_filename)

  #return dataframe
  norm_preds = pd.DataFrame(norm_preds)
  mse = pd.DataFrame(mse)

  return norm_preds, mse

## Loop through Stocks

In [ ]:
#Read in Data
prices_df = pd.read_csv("/content/drive/My Drive/HODL_Project/closing_prices.csv")

In [ ]:
#Normalize Data
normalized_df = prices_df.copy()
min_series = prices_df.iloc[:, 1:].min()
max_series = prices_df.iloc[:, 1:].max()
normalized_df.iloc[:, 1:] = (prices_df.iloc[:, 1:] - min_series) / (max_series - min_series)

In [ ]:
#Update each time
last_pred_date = '1/2/2024'

#Initial Split
train_df = normalized_df[normalized_df['Date'] < last_pred_date]
test_df = normalized_df[normalized_df['Date'] >= last_pred_date]

#date range
date_range = test_df['Date'].tolist()

In [ ]:
#Intialize DataFrames - First Time Only
norm_preds_df = pd.DataFrame(data=date_range[1:], columns = ['Date'])
mse_df = pd.DataFrame(data=date_range[1:], columns = ['Date'])

In [ ]:
#loop through all stocks
for j in range(start_index, prices_df.shape[1]):
  print(j)
  #Load Model
  model_filename = f'/content/drive/My Drive/HODL_Project/LSTM_Models/model_column_{j}.h5'
  model = load_model(model_filename)
  model.compile(optimizer='adam', loss='mse')

  #Series for jth stock
  train_series = train_df.iloc[:,j]
  test_series = test_df.iloc[:,j]

  #Update the model
  norm_preds, mse = update_model(model, train_series, test_series)

  norm_preds_df = pd.concat([norm_preds_df, norm_preds], axis=1)
  mse_df = pd.concat([mse_df, mse], axis=1)

  #Save DataFrames
  norm_preds_df.to_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/norm_preds.csv', index=False)
  mse_df.to_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/mse.csv', index=False)


In [ ]:
#Once all stocks are have run - update columns
prices_df_cols = list(prices_df.columns)
norm_preds_df.columns = prices_df_cols
mse_df.columns = prices_df_cols

In [ ]:
# De-normalize predictions
preds_df = pd.DataFrame(data=date_range[1:], columns = ['Date'])

for j in range(1, norm_preds_df.shape[1]):
  de_norm = norm_preds_df.iloc[:,j] * (max_series[j-1] - min_series[j-1]) + min_series[j-1]
  preds_df = pd.concat([preds_df, de_norm], axis=1)

#Rename columns
preds_df.columns = prices_df_cols

In [ ]:
#save csv with col names
norm_preds_df.to_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/norm_preds.csv', index=False)
preds_df.to_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/preds.csv', index=False)
mse_df.to_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/mse.csv', index=False)


## Evaluation Measures

In [ ]:
#Read in Files if not already loaded - commented out because assumed loaded
prices_df = pd.read_csv("/content/drive/My Drive/HODL_Project/closing_prices.csv")
norm_preds_df = pd.read_csv("/content/drive/My Drive/HODL_Project/Prediction_DF/norm_preds.csv")
preds_df = pd.read_csv("/content/drive/My Drive/HODL_Project/Prediction_DF/preds.csv")
mse_df = pd.read_csv("/content/drive/My Drive/HODL_Project/Prediction_DF/mse.csv")

#Normalize Data
normalized_df = prices_df.copy()
min_series = prices_df.iloc[:, 1:].min()
max_series = prices_df.iloc[:, 1:].max()
normalized_df.iloc[:, 1:] = (prices_df.iloc[:, 1:] - min_series) / (max_series - min_series)

#Assure last_pred_date & price_df_cols is defined
last_pred_date = pd.to_datetime('1/2/2024').tz_localize('UTC')
prices_df_cols = list(prices_df.columns)

In [ ]:
#Get train DF
test_df_norm = normalized_df[pd.to_datetime(normalized_df['Date']) > last_pred_date]
test_df = prices_df[pd.to_datetime(prices_df['Date'])  > last_pred_date]

In [ ]:
#Define date_range
date_range = test_df['Date'].tolist()

In [ ]:
test_df_norm.head()

### Mean Absolute Error

In [ ]:
#Calc MAE for all stocks

mae_df = pd.DataFrame(data=date_range[1:], columns = ['Date'])

for j in range(1, preds_df.shape[1]):
  mae_list = []
  for i in range(0, preds_df.shape[0]):
    mae = abs(preds_df.iloc[i,j] - test_df.iloc[i,j])
    mae_list.append(mae)

  mae_df_j = pd.DataFrame(mae_list)
  mae_df = pd.concat([mae_df, mae_df_j], axis=1)

mae_df.columns = prices_df_cols

In [ ]:
#Rolling MAE - does not include that date
rolling_mae_7day  = pd.DataFrame(data=date_range[2:], columns = ['Date'])
for j in range(1, mae_df.shape[1]):
  col = mae_df.iloc[:,j].rolling(window=7).mean()
  rolling_mae_7day = pd.concat([rolling_mae_7day, col], axis=1)

rolling_mae_7day.columns = prices_df_cols
rolling_mae_7day = rolling_mae_7day.iloc[:-1]

### Forecasted Change & MDE

In [ ]:
#Initalize DFs
#Forecasted Change
delta_df = pd.DataFrame(data=date_range[1:], columns = ['Date'])
#Forecasted % Change
delta_perc_df = pd.DataFrame(data=date_range[1:], columns = ['Date'])
#Actual Price Change
actual_delta_df = pd.DataFrame(data=date_range[1:], columns = ['Date'])
#If actual and predicted change have same sign
directional_error_df = pd.DataFrame(data=date_range[1:], columns = ['Date'])


In [ ]:
for j in range(1, preds_df.shape[1]):
  #Initialize Lists
  delta_list = []
  delta_perc_list = []
  actual_delta_list = []
  directional_error_list = []
  for i in range(0, preds_df.shape[0]):
    #Get forecasted change & % change
    delta = preds_df.iloc[i,j] - test_df.iloc[i,j] #same i b/c test_df starts on 1/2 but pred_df on 1/3
    delta_perc = delta / test_df.iloc[i,j]
    #Append to list
    delta_list.append(delta)
    delta_perc_list.append(delta_perc)

    #Get Actual Change
    actual_delta = test_df.iloc[i+1,j] - test_df.iloc[i,j]
    actual_delta_list.append(actual_delta)

    #Directional Error
    #Calc directional error : 0 if wrong 1 if right
    if actual_delta > 0 and delta > 0:
      directional_error = 1
    elif actual_delta < 0 and delta < 0:
      directional_error = 1
    else:
      directional_error = 0
    directional_error_list.append(directional_error)

  #Join dfs
  delta_df = pd.concat([delta_df, pd.DataFrame(delta_list)], axis=1)
  delta_perc_df = pd.concat([delta_perc_df, pd.DataFrame(delta_perc_list)], axis=1)
  actual_delta_df = pd.concat([actual_delta_df, pd.DataFrame(actual_delta_list)], axis=1)
  directional_error_df = pd.concat([directional_error_df, pd.DataFrame(directional_error_list)], axis=1)


In [ ]:
#Rename Columns
delta_df.columns = prices_df_cols
delta_perc_df.columns = prices_df_cols
actual_delta_df.columns = prices_df_cols
directional_error_df.columns = prices_df_cols

In [ ]:
#Make a 7 day rolling MDE (output is a percentage)
rolling_mde_7day  = pd.DataFrame(data=date_range[2:], columns = ['Date'])
for j in range(1, directional_error_df.shape[1]):
  col = directional_error_df.iloc[:,j].rolling(window=7).mean() * 100
  rolling_mde_7day = pd.concat([rolling_mde_7day, col], axis=1)

rolling_mde_7day.columns = prices_df_cols
rolling_mde_7day = rolling_mde_7day.iloc[:-1]

### Save dfs needs for agent

In [ ]:
delta_df.to_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/delta_df.csv', index = False)
rolling_mde_7day.to_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/rolling_mde_7day.csv', index = False)
delta_perc_df.to_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/delta_perc_df.csv', index = False)
rolling_mae_7day.to_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/rolling_mae_7day.csv', index = False)

## See how 1 stock performs DFs

In [ ]:
stock = 'AAPL'
stock_df = test_df[['Date', stock]]
stock_df.rename(columns = {stock:'Actual'}, inplace = True)
stock_df = stock_df.merge(preds_df[['Date', stock]],
                        on = 'Date',
                        how = 'inner')
stock_df.rename(columns = {stock:'Predicted'}, inplace = True)

In [ ]:
plt.plot(pd.to_datetime(stock_df['Date']), stock_df['Actual'])
plt.plot(pd.to_datetime(stock_df['Date']), stock_df['Predicted'])
plt.legend(['Actual', 'Predicted'])
plt.xticks(rotation=45)
plt.tight_layout()
plt.title(stock)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

## Calc Baseline MSE & MAE

In [ ]:
baseline_mse_df = pd.DataFrame(data=date_range[1:], columns = ['Date'])

In [ ]:
for j in range(1, normalized_df.shape[1]):

  mse_list = []

  for i in range(1, normalized_df.shape[0]):
    mse = np.square(normalized_df.iloc[i-1,j] - normalized_df.iloc[i,j])
    mse_list.append(mse)

  baseline_mse_df = pd.concat([baseline_mse_df, pd.DataFrame(data=mse_list, columns = [prices_df_cols[j]])], axis=1)

In [ ]:
baseline_mae_df = pd.DataFrame(data=date_range[1:], columns = ['Date'])

for j in range(1, test_df.shape[1]):

  mae_list = []

  for i in range(1, test_df.shape[0]):
    mae = abs(test_df.iloc[i,j] - test_df.iloc[i-1,j])
    mae_list.append(mae)

  baseline_mae_df = pd.concat([baseline_mae_df, pd.DataFrame(data=mae_list, columns = [prices_df_cols[j]])], axis=1)

## Compare NMSE to baseline

In [ ]:
#LSTM
mean_mse = mse_df.mean()
average_mse_df = mean_mse.to_frame(name='Average MSE')

# Reset the index to make the stock names a column
average_mse_df.reset_index(inplace=True)
average_mse_df.rename(columns={'index': 'Stock'}, inplace=True)

In [ ]:
#Make Histogram
mse_avg_data = average_mse_df['Average MSE']
mse_avg_data = np.clip(mse_avg_data, 0, 0.05)
bins = np.arange(start=min(mse_avg_data), stop=0.05, step=0.0025)  # Adjust step for finer or coarser bins
bins = np.append(bins, np.max(mse_avg_data))

plt.hist(mse_avg_data, bins=bins, edgecolor='black')
plt.title('LSTM: NMSE Distribution')
plt.xlabel('Avg MSE')
plt.ylabel('Frequency')

# Displaying the histogram
plt.show()

In [ ]:
#Baseline
mean_baseline_mse = baseline_mse_df.mean()
average_baseline_mse_df = mean_baseline_mse.to_frame(name='Average MSE')

# Reset the index to make the stock names a column
average_baseline_mse_df.reset_index(inplace=True)
average_baseline_mse_df.rename(columns={'index': 'Stock'}, inplace=True)

In [ ]:
#Make a histogram of performance
b_mse_avg_data = average_baseline_mse_df['Average MSE']
b_mse_avg_data = np.clip(b_mse_avg_data, 0, 0.05)
bins = np.arange(start=min(b_mse_avg_data), stop=0.05, step=0.0025)  # Adjust step for finer or coarser bins
#bins = np.append(bins, np.max(b_mse_avg_data))

plt.hist(b_mse_avg_data, bins=bins, edgecolor='black')
plt.title('Baseline: NMSE Distribution')
plt.xlabel('Avg MSE')
plt.ylabel('Companies')

# Displaying the histogram
plt.show()

In [ ]:
#Determine how many stocks the LSTM performs better for
better = 0

for i in range(1, len(mse_avg_data)):
  if mse_avg_data[i] < b_mse_avg_data[i]:
    better += 1

print(better)

## Compare MAE to Baseline

In [ ]:
#Calc Average Baseline MAE
mean_baseline_mae = baseline_mae_df.mean()
average_baseline_mae_df = mean_baseline_mae.to_frame(name='Baseline MAE')
# Reset the index to make the stock names a column
average_baseline_mae_df.reset_index(inplace=True)
average_baseline_mae_df.rename(columns={'index': 'Stock'}, inplace=True)

In [ ]:
#Calc Average LSTM MAE
mean_mae = mae_df.mean()
average_mae_df = mean_mae.to_frame(name='Predicted MAE')
average_mae_df.reset_index(inplace=True)
average_mae_df.rename(columns={'index': 'Stock'}, inplace=True)

In [ ]:
#Join the DFs
average_mae_df = average_mae_df.merge(average_baseline_mae_df, on='Stock')

In [ ]:
#Calc difference
average_mae_df['Dif'] = average_mae_df['Predicted MAE'] - average_mae_df['Baseline MAE']

In [ ]:
#Determine how many are negative (baseline performs better)
average_mae_df['Negative Dif'] = average_mae_df['Dif'] < 0
average_mae_df['Negative Dif'].sum()

## Mean Directional Error in 2024

In [ ]:
avg_mde = directional_error_df.mean()
average_mde_df = avg_mde.to_frame(name='Average MDE')

# Reset the index to make the stock names a column
average_mde_df.reset_index(inplace=True)
average_mde_df.rename(columns={'index': 'Stock'}, inplace=True)

In [ ]:
#Average across all stocks
average_mde_df['Average MDE'].mean()

In [ ]:
#Make Histogram
b_mde_avg_data = average_mde_df['Average MDE']
bins = np.arange(start=0, stop=1, step=0.1)  # Adjust step for finer or coarser bins
#bins = np.append(bins, np.max(b_mse_avg_data))

plt.hist(b_mde_avg_data, bins=bins, edgecolor='black')
plt.title('LSTM: MDE Distribution')
plt.xlabel('Avg MDE')
plt.ylabel('Companies')

# Displaying the histogram
plt.show()

# Garch Model Evaluation

In [ ]:
df = pd.read_csv("/content/drive/My Drive/HODL_Project/closing_prices.csv")
df['Date'] = pd.to_datetime(df['Date'], utc = True)
df

In [ ]:
series0 = df[(df['Date'].dt.year == 2023) | (df['Date'].dt.year == 2022)]
#467(2022-02-22 to 2023-12-29) * 498(Date + 497 Stocks)

df_2024 = df[df['Date'].dt.year == 2024]
#35 rows(index:467-501, 2024-01-02 to 2024-02-21) × 498 columns

In [ ]:
date_range = df[df['Date'] >= '2024-01-02']['Date']
date_range = date_range.tolist()
len(date_range)

#### Initialize dataframes for storing predicted prices, MSEs, and volatilities (only run once)

In [ ]:
stock_names = list(df.columns)[1:]
num_rows = 36

preds_df = pd.DataFrame(columns=['Date'])
preds_df['Date'] = date_range
mse_df = pd.DataFrame(columns=['Date'])
mse_df['Date'] = date_range
volatility_df = pd.DataFrame(columns=['Date'])
volatility_df['Date'] = date_range

for stock_name in stock_names:
    preds_df[stock_name] = [None] * (num_rows-1)
    mse_df[stock_name] = [None] * (num_rows-1)
    volatility_df[stock_name] = [None] * (num_rows-1)
#35 rows × 498 columns

# Save the initialized DataFrames to CSV files (only run once)
#preds_df.to_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/garch_preds.csv', index=False)
#mse_df.to_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/garch_mse.csv', index=False)
#volatility_df.to_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/garch_volatility.csv', index=False)

#### Calculate log returns
shifts all the values in the DataFrame df except for the first column (iloc[:, 1:]) down by one row.


*   Each value in the DataFrame is moved down by one row.
*  The first row becomes NaN (missing value) because there's no value to shift into its place.
*   The last row is discarded because it's shifted out of the DataFrame's range.


In [ ]:
def log_returns(df):
  #skip the 0th col (the Date col), subset from the 1st col
  log_returns = np.log(df.iloc[:, 1:] / df.iloc[:, 1:].shift(1))
  log_returns = log_returns.dropna()
  return log_returns

### For all stocks from 2024-01-02 to 2024-02-21

In [ ]:
preds_df = pd.read_csv("/content/drive/My Drive/HODL_Project/Prediction_DF/garch_preds.csv")
mse_df = pd.read_csv("/content/drive/My Drive/HODL_Project/Prediction_DF/garch_mse.csv")
volatility_df = pd.read_csv("/content/drive/My Drive/HODL_Project/Prediction_DF/garch_volatility.csv")

In [ ]:
#base_save_path = '/content/drive/My Drive/HODL_Project/Updated_GARCH/'
for stock in range(0, 497):
  mse_column = []  #list to store MSE values for the current stock
  next_day_price_pred_column = []  #list to store next day price predictions for the current stock
  volatility_column = []
  for time in range(0, 35):
    #when time=0: initial models
    series_df = pd.concat([series0, df_2024.head(time)])
    series_log_returns = log_returns(series_df).iloc[:,stock]

    # Fit a GARCH(1,1) model
    #uses the most recent observation and the previous observation to calculate the conditional variance at each time step.
    #effective rolling window size is 2.
    model = arch_model(series_log_returns, vol='Garch', p=1, q=1, mean='constant', dist='Normal', rescale=False)
    results = model.fit(disp='off')

    ###########################################################################
    # Predict the next day's returns
    pred_returns = results.forecast(horizon=1).mean.iloc[-1].values[0]

    # extract the last available/known price
    last_available_price = series_df.iloc[-1,stock+1]

    # Use the last available price to predict the next day's price
    next_day_price_pred = last_available_price * np.exp(pred_returns)
    next_day_price_pred_column.append(next_day_price_pred)

    # Calculate mean squared error (MSE) for price prediction
    actual_next_day_price = df_2024.iloc[time,stock+1]
    price_mse = (next_day_price_pred - actual_next_day_price) ** 2
    mse_column.append(price_mse)
    ###########################################################################
    # Extract the forecasted conditional variance
    forecasted_conditional_variance = results.forecast(horizon=1).variance.iloc[-1]
    # Calculate volatility as the square root of conditional variance
    volatility = np.sqrt(forecasted_conditional_variance)
    volatility_column.append(volatility)
    ###########################################################################
    # # Construct the save path for the model
    # model_save_path = os.path.join(base_save_path, f'garch_model_stock{stock+1}_day{time+1}.pkl')

    # # Save the model using pickle
    # with open(model_save_path, 'wb') as f:
    #     pickle.dump(results, f)

    # print(f"Model saved to {model_save_path}")
    ###########################################################################
    #print(f'Stock{stock+1}_Day{time+1}')

  # Add the next day price predictions and MSE values to the corresponding cells in the DataFrames
  preds_df.iloc[:, stock+1] = next_day_price_pred_column
  mse_df.iloc[:, stock+1] = mse_column
  volatility_df.iloc[:, stock+1] = volatility_column
  print(f'Stock{stock+1}')

# Save the updated DataFrames back to the CSV files
#preds_df.to_csv("/content/drive/My Drive/HODL_Project/Prediction_DF/garch_preds.csv", index=False)
#mse_df.to_csv("/content/drive/My Drive/HODL_Project/Prediction_DF/garch_mse.csv", index=False)
#volatility_df.to_csv("/content/drive/My Drive/HODL_Project/Prediction_DF/garch_volatility.csv", index=False)

^^

Takes ~15min to run with model model saving

Takes ~7min to run without model saving

### Plots - predicted vs. actual price

In [ ]:
preds_df = pd.read_csv("/content/drive/My Drive/HODL_Project/Prediction_DF/garch_preds.csv")

In [ ]:
# GOOGL
actual_prices = df_2024['GOOGL'].tolist()
predicted_prices = preds_df['GOOGL'].tolist()
dates = date_range

dates = pd.to_datetime(dates)

plt.plot(dates, actual_prices, label='Actual Prices')
plt.plot(dates, predicted_prices, label='Predicted Prices')

plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Actual vs Predicted Prices for GOOGL')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.legend()
plt.show()

In [ ]:
# AMZN
actual_prices = df_2024['AMZN'].tolist()
predicted_prices = preds_df['AMZN'].tolist()
dates = date_range

dates = pd.to_datetime(dates)

plt.plot(dates, actual_prices, label='Actual Prices')
plt.plot(dates, predicted_prices, label='Predicted Prices')

plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Actual vs Predicted Prices for AMZN')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.legend()
plt.show()

In [ ]:
# BKNG
actual_prices = df_2024['BKNG'].tolist()
predicted_prices = preds_df['BKNG'].tolist()
dates = date_range

dates = pd.to_datetime(dates)

plt.plot(dates, actual_prices, label='Actual Prices')
plt.plot(dates, predicted_prices, label='Predicted Prices')

plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Actual vs Predicted Prices for BKNG')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.legend()
plt.show()

### Plots - predicted vs. actual volatility
Volatility: measure of the amount of uncertainty or risk involved in the size of changes in a stock's value.

* Calculate actual volatility (in 2024) from the historical closing prices data

  * uses the most recent observation and the previous observation to calculate the conditional variance

  * effective rolling window size = 2

In [ ]:
#subset historical closing prices to start from one day before the first day in 2024
closing_prices_2024 = pd.concat([series0.tail(2), df_2024], ignore_index=True)

#(don't need to drop the first "Date" col because the log_returns(df) function handles that)

#calculate log returns (using function defined earlier)
calculated_log_returns = log_returns(closing_prices_2024)

# Calculate rolling variance of log returns using only previous day info
rolling_variance = calculated_log_returns.rolling(window=2).var()

# Calculate volatility as the square root of rolling variance
actual_volatility = np.sqrt(rolling_variance)

In [ ]:
#drop the first row which becomes NaNs after calculating the log_returns
actual_volatility = actual_volatility.dropna()

# Add a new row at the end with NaN values (referring to the volatility on the last day)
#actual_volatility.loc[len(actual_volatility)] = [np.nan] * len(actual_volatility.columns)

# Insert the "Date" column at the beginning of the DataFrame
actual_volatility.insert(0, "Date", date_range)

#actual_volatility.to_csv("/content/drive/My Drive/HODL_Project/Prediction_DF/garch_actual_volatility.csv", index=False)

In [ ]:
volatility_df = pd.read_csv("/content/drive/My Drive/HODL_Project/Prediction_DF/garch_volatility.csv")
actual_volatility = pd.read_csv("/content/drive/My Drive/HODL_Project/Prediction_DF/garch_actual_volatility.csv")

date_range = df[df['Date'] >= '2024-01-02']['Date']
date_range = date_range.tolist()

dates = date_range
dates = pd.to_datetime(dates)

In [ ]:
# GOOGL
pred_vol = volatility_df['GOOGL'].tolist()
actual_vol = actual_volatility['GOOGL'].tolist()

plt.plot(dates, pred_vol, label='Predicted Volatility')
plt.plot(dates, actual_vol, label='Actual Volatility')

plt.xlabel('Date')
plt.ylabel('Volatility')
plt.title('Predicted and Actual Volatility for GOOGL')
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
# AMZN
pred_vol_amzn = volatility_df['AMZN'].tolist()
actual_vol_amzn = actual_volatility['AMZN'].tolist()

plt.plot(dates, pred_vol_amzn, label='Predicted Volatility')
plt.plot(dates, actual_vol_amzn, label='Actual Volatility')

plt.xlabel('Date')
plt.ylabel('Volatility')
plt.title('Predicted and Actual Volatility for AMZN')
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
# BKNG
pred_vol_bkng = volatility_df['BKNG'].tolist()
actual_vol_bkng = actual_volatility['BKNG'].tolist()

plt.plot(dates, pred_vol_bkng, label='Predicted Volatility')
plt.plot(dates, actual_vol_bkng, label='Actual Volatility')

plt.xlabel('Date')
plt.ylabel('Volatility')
plt.title('Predicted and Actual Volatility for BKNG')
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
# AAPL
pred_vol_aapl = volatility_df['AAPL'].tolist()
actual_vol_aapl = actual_volatility['AAPL'].tolist()

plt.plot(dates, pred_vol_aapl, label='Predicted Volatility')
plt.plot(dates, actual_vol_aapl, label='Actual Volatility')

plt.xlabel('Date')
plt.ylabel('Volatility')
plt.title('Predicted and Actual Volatility for AAPL')
plt.xticks(rotation=45)
plt.legend()
plt.show()

# Output for Agent

## Get Company Names (Wikipedia)

In [ ]:
#Function to Get company name
def fetch_sp500_info():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)  # This parses all the tables in webpages
    sp500_table = tables[0]  # Assuming the first table is the S&P 500 list
    sp500_table = sp500_table[['Symbol', 'Security']]  # Adjust column name if necessary
    return sp500_table

In [ ]:
#Save company name in a table
sp500_table = fetch_sp500_info()

## Make sure all data is present

In [ ]:
#Read in all data (if needed)
prices_df = pd.read_csv("/content/drive/My Drive/HODL_Project/closing_prices.csv")
actual_vol_df = pd.read_csv("/content/drive/My Drive/HODL_Project/Prediction_DF/garch_actual_volatility.csv")
pred_vol_df = pd.read_csv("/content/drive/My Drive/HODL_Project/Prediction_DF/garch_volatility.csv")
preds_df = pd.read_csv("/content/drive/My Drive/HODL_Project/Prediction_DF/preds.csv")
delta_df = pd.read_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/delta_df.csv')
rolling_mde_7day = pd.read_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/rolling_mde_7day.csv')
delta_perc_df = pd.read_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/delta_perc_df.csv')
rolling_mae_7day = pd.read_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/rolling_mae_7day.csv')

## Calc Garch Model % Change

In [ ]:
#Change in Volitility DF - Actual starts 1/2, pred starts 1/3
#Check calculation is right
perc_change_pred_vol_df = pd.DataFrame(data=date_range[1:], columns = ['Date'])

for j in range(1, pred_vol_df.shape[1]):
  vol_change_list = []
  for i in range(0, pred_vol_df.shape[0]-1):
    val = (pred_vol_df.iloc[i+1,j] - actual_vol_df.iloc[i,j]) / actual_vol_df.iloc[i,j]
    vol_change_list.append(val)
  col = pd.DataFrame(vol_change_list)
  perc_change_pred_vol_df = pd.concat([perc_change_pred_vol_df, col], axis=1)

perc_change_pred_vol_df.columns = prices_df_cols

## Extract Relevant Rows from DFs

In [ ]:
#Get Last Day Predictions

#Last actual - 2 days before:
last_price = prices_df.iloc[-2,1:]
last_actual_vol = actual_vol_df.iloc[-2,1:]

#Preds - Last Row
next_pred = preds_df.iloc[-1,1:]
last_pred_vol = pred_vol_df.iloc[-1,1:]

#Change - Last Row
last_pred_change = delta_df.iloc[-1,1:]
last_pred_change_perc = delta_perc_df.iloc[-1,1:]
last_perc_vol_perc = perc_change_pred_vol_df.iloc[-1,1:]  * 100

#Rolling Errors - Last Row based on code design (excludes the day itself)
last_avg_mse = rolling_mae_7day.iloc[-1,1:]
last_avg_mde = rolling_mde_7day.iloc[-1,1:]

## Make DF

In [ ]:
### May need to update based on Yutong's Code #####
#Make Dictionary
output_df = {'Ticker' : prices_df_cols[1:],
             'current_price': last_price,
             'forecasted_price' : next_pred,
             'forecasted_price_change':last_pred_change,
             'percent_change': last_pred_change_perc * 100,
             'current_volatility': last_actual_vol,
             'forecasted_volatility': last_pred_vol,
             'forecasted_volatility_change': last_perc_vol_perc,
             'model_error_last_week' : last_avg_mse,
             'model_mda_last_week':last_avg_mde}

#Make DF
output_df = pd.DataFrame(data=output_df)

In [ ]:
#Join with SP500 table for company name
output_df = output_df.merge(sp500_table, left_on='Ticker', right_on = 'Symbol', how='left')

In [ ]:
#Reorder cols so company name is second
cols = list(output_df.columns)
new_order = [cols[0]] + [cols[-1]] + cols[1:-1]
output_df = output_df[new_order]
output_df.head()

In [ ]:
output_df = output_df.drop(columns=['Symbol'])
output_df = output_df.rename(columns={'Security': 'Stock'})

In [ ]:
output_df['Ticker'] = output_df['Ticker'].str.lower()
output_df['Stock'] = output_df['Stock'].str.lower()
output_df.tail()

In [ ]:
#Save Output
output_df.to_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/output_df.csv', index=False)

# Slot Filing

Link to Github: xxx

In [ ]:
# Download the "HODL" library
!wget -q 'https://www.dropbox.com/s/4rdgil1epnvgitf/HODL.py'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from zipfile import ZipFile

import tensorflow as tf
from tensorflow import keras

from HODL import TransformerEncoder, PositionalEmbedding

##################### CONSTANTS #####################
keras.utils.set_random_seed(2024)
MAX_QUERY_LENGTH = 50  #size of input
EMBED_DIM = 512  #dimension of embeddings
DENSE_DIM = 128
NUM_HEADS = 8  #number of multi-attention heads
DENSE_UNITS = 128  #num nodes in hidden layer
BATCH_SIZE = 64  #batch size for training transformer
EPOCHS = 10  #epochs for training transformer

# Read training data
test_df = pd.read_csv("/content/drive/My Drive/HODL_Project/testing_data.csv")
train_df = pd.read_csv("/content/drive/My Drive/HODL_Project/training_data.csv")

train_query = train_df['query'].values
train_slotfilling = train_df['slot filling'].values
test_query = test_df['query'].values
test_slotfilling = test_df['slot filling'].values
#####################################################

# CREATE VECTORIZER (QUERY & SLOTS)
vectorize_query_text = keras.layers.TextVectorization(
    max_tokens=None,  #no maximum vocabulary
    output_sequence_length=MAX_QUERY_LENGTH,  #pad or truncate output to value
    output_mode="int",  #vector has index of vocabulary
    standardize="lower_and_strip_punctuation",  #convert input to lowercase and rmv punctuation
    split="whitespace",  #split values based on whitespace
    ngrams=1  #only look at whole words
)
vectorize_slot_text = keras.layers.TextVectorization(
    max_tokens=None,  #no maximum vocabulary
    output_sequence_length=MAX_QUERY_LENGTH,
    output_mode="int",  #vector has index of vocabulary
    standardize="lower",  #convert input to lowercase [can't do punctuation b/c of dashes]
    split="whitespace",  #split values based on whitespace
    ngrams=1  #only look at whole words
)

# CREATE VOCABULARY AND VECTORIZED TRAINING DATA
vectorize_query_text.adapt(train_query)  #build vocabulary
query_train = vectorize_query_text(train_query)  #vectorized training queries
query_test = vectorize_query_text(test_query)  #vectorized testing queries
QUERY_VOCAB_SIZE = vectorize_query_text.vocabulary_size() #total vocabulary of queries

vectorize_slot_text.adapt(train_slotfilling)  #build slot vocabulary
slots_train = vectorize_slot_text(train_slotfilling)  #vectorized training slots
slots_test = vectorize_slot_text(test_slotfilling)  #vectorized testing slots
SLOT_VOCAB_SIZE = vectorize_slot_text.vocabulary_size()  #total vocabulary of slots

# BUILD KERAS MODEL
inputs = keras.Input(shape=(MAX_QUERY_LENGTH,))
embedding = PositionalEmbedding(MAX_QUERY_LENGTH,
                                QUERY_VOCAB_SIZE,
                                EMBED_DIM)
x = embedding(inputs)
encoder_out = TransformerEncoder(EMBED_DIM,
                                 DENSE_DIM,
                                 NUM_HEADS)(x)
x = keras.layers.Dense(DENSE_UNITS, activation="relu", name="Dense_Layer")(encoder_out)
x = keras.layers.Dropout(0.25, name="Dropout_Layer")(x)
outputs = keras.layers.Dense(SLOT_VOCAB_SIZE, activation="softmax", name="Softmax_Layer")(x)

model = keras.Model(inputs, outputs)
print()
print(model.summary())
print()

# TRAIN KERAS MODEL
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["sparse_categorical_accuracy"])
history = model.fit(query_train, slots_train,
                 batch_size=BATCH_SIZE,
                 epochs=EPOCHS)

# OUT-OF-SAMPLE TESTING
model.evaluate(query_test, slots_test)

# SAVE MODEL
filename = '/content/drive/My Drive/HODL_Project/sql_transformer.keras'
model.save(filename)
ZipFile('/content/drive/My Drive/HODL_Project/model_save.zip', mode='w').write(filename)

# EVALUATING SLOT ACCURACY
def slot_filling_accuracy(actual, predicted, only_slots=False):
    not_padding = np.not_equal(actual, 0) #+ np.not_equal(predicted, 0)

    if only_slots:
        non_slot_token = vectorize_slot_text(['O']).numpy()[0, 0]
        slots = np.not_equal(actual, non_slot_token)
        correct_predictions = np.equal(actual, predicted)[not_padding * slots]
    else:
        correct_predictions = np.equal(actual, predicted)[not_padding]

    sample_length = len(correct_predictions)

    weights = np.ones(sample_length)

    return np.dot(correct_predictions, weights) / sample_length

predicted = np.argmax(model.predict(query_test), axis=-1).reshape(-1)
actual = slots_test.numpy().reshape(-1)

acc = slot_filling_accuracy(actual, predicted, only_slots=False)
acc_slots = slot_filling_accuracy(actual, predicted, only_slots=True)

print(f'Accuracy = {acc:.3f}')
print(f'Accuracy on slots = {acc_slots:.3f}')

# TEST-SET EVALUATION
def predict_slots_query(query, model, query_vectorizer, slot_vectorizer):
    sentence = query_vectorizer([query])

    prediction = np.argmax(model.predict(sentence), axis=-1)[0]

    inverse_vocab = dict(enumerate(slot_vectorizer.get_vocabulary()))
    decoded_prediction = " ".join(inverse_vocab[int(i)] for i in prediction)
    return decoded_prediction

for example, answer in zip(test_query, test_slotfilling):
    print()
    print("Query:\n", example)
    print("Answer:\n", answer)
    print("Prediction:\n", predict_slots_query(example,
                                               model,
                                               vectorize_query_text,
                                               vectorize_slot_text))
    print()

## Slot Parser Function

In [ ]:
def SlotParser(slot_filling, prompt, stock_data):
    slot_filling = slot_filling.strip()  #strip any whitespace from slot_filling return
    # initialize aspects of the SQL query
    slots = {'select': [],
             'order': [],
             'limit': None}

    # for word (token) in prompt
    for word in prompt.split(" "):
        if word.isnumeric():  #if we found a number
            slots['limit'] = int(word)  #assume number relates to LIMIT (# of rows to display)

    # for slot (token) in output
    for token in slot_filling.split(" "):
        if (token != "o") and (token != "O"):  #if not a null slot
            if 'select-' in token:  #if a select slot
                if token not in slots['select']:  #if we haven't already added it
                    slots['select'].append(token)  #then include in SELECT statement
            elif 'order-by' in token:  #if an order-by slot
                if token not in slots['order']:  #if we haven't already added it
                    slots['order'].append(token)  #then include in ORDER BY statement

    if len(slots['select']) == 0:  #if we haven't selected a colummn
        columns = list(stock_data.columns)  #assume all
    else:  #if we have selected >= 1 column
        columns = [x.split("-")[1] for x in slots['select']]  #access column names, format them

    if len(slots['order']) == 0:  #if we haven't ordered a column
        if 'stock' not in columns:
            columns.insert(0, 'stock')  #insert at beginning
        order_cols = ['stock']  #assume we order by stock
        order_ascending = [True]  #assum ascending order
    else:  #if there is order statement
        order_cols = [x.split("-")[2] for x in slots['order']]  #format
        order_ascending = [True if x.split("-")[-1] == 'asc' else False for x in slots['order']]  #format

    if slots['limit'] == None:  #if we don't have a number
        limit = len(stock_data)  #assume we want all rows
    else:  #if there is a limit number
        limit = int(slots['limit'])  #make sure to return that many rows

    for col in order_cols:  #for every ordering column
        if col not in columns:  #check if it is an accessible column
            columns.append(col)  #if not, add it

    pandas_query = stock_data[columns]  #use only selected columns
    pandas_query = pandas_query.sort_values(by=order_cols,  #columns to sort
                                            ascending=order_ascending,  #boolean list of asc/desc
                                            ignore_index=True)
    pandas_query = pandas_query.head(limit)  #LIMIT statement

    all_data = stock_data.sort_values(by=order_cols,
                                      ascending=order_ascending,
                                      ignore_index=True)  #include just in case
    # print(pandas_query)  #print the dataframe

    #formatting SQL statements
    SELECT = f"SELECT {', '.join(columns)}"
    FROM = "FROM stock_data"
    ORDER_BY = f"ORDER BY {', '.join((str(x.split('-')[2])+' '+str(x.split('-')[-1].upper())) for x in slots['order'])}"
    LIMIT = f"LIMIT {slots['limit']}"

    #checking for errors
    if len(slots['select']) == 0:  #empty slots
        SQL_QUERY = "SELECT *\n FROM stock_data"
    elif (len(slots['order']) == 0) and (slots['limit'] == None):  #no sort or limit statement
        SQL_QUERY = f"{SELECT}\n{FROM}"
    elif len(slots['order']) == 0:  #no order clause
        SQL_QUERY = f"{SELECT}\n{FROM}\n{LIMIT}"
    elif slots['limit'] == None:  #no limit clause
        SQL_QUERY = f"{SELECT}\n{FROM}\n{ORDER_BY}"
    else:
        SQL_QUERY = f"{SELECT}\n{FROM}\n{ORDER_BY}\n{LIMIT}"

    print(SQL_QUERY)  #print SQL statement

    return pandas_query, SQL_QUERY, all_data

## Slot Filling Evaluation

In [ ]:
import pandas as pd

example_slots = [
    "o select-stock o o o o o order-by-forecasted_price_change-desc select-forecasted_price o o o o",
    "o select-stock o select-forecasted_volatility o select-forecasted_volatility o o o order-by-forecasted_volatility-desc select-forecasted_volatility o o o",
    "o select-stock o o o o o o o order-by-percent_change-asc select-percent_change select-percent_change o o",
    "o select-ticker o o o o o order-by-forecasted_price-desc select-forecasted_price o order-by-forecasted_volatility-asc select-forecasted_volatility"
]
example_prompts = [
    "What 5 stocks are expected to have the highest increase in price for tomorrow?",
    "What 10 stocks are forecasted or predicted to have the highest volatility in their price?",
    "What 2 stocks are forecasted or predicted to have the lowest percent change in price?",
    "What 8 tickers are predicted to have the highest price and lowest volatility"
]
data = pd.read_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/output_df.csv')

for (slot, prompt) in zip(example_slots, example_prompts):
    print(slot)
    print(prompt)
    SlotParser(slot, prompt, data)
    print()

## Web Application

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from zipfile import ZipFile
import re

from HODL import TransformerEncoder, PositionalEmbedding

##################### CONSTANTS #####################
keras.utils.set_random_seed(2024)
MAX_QUERY_LENGTH = 50  #size of input

# Read training data
train_df = pd.read_csv("/content/drive/My Drive/HODL_Project/training_data.csv")

train_query = train_df['query'].values
train_slotfilling = train_df['slot filling'].values

transformer_model = keras.models.load_model("/content/drive/My Drive/HODL_Project/sql_transformer.keras", custom_objects={
    "TransformerEncoder": TransformerEncoder,
    "PositionalEmbedding": PositionalEmbedding,
})


stock_data = pd.read_csv('/content/drive/My Drive/HODL_Project/Prediction_DF/output_df.csv')
#####################################################

# CREATE VECTORIZER (QUERY & SLOTS)
vectorize_query_text = keras.layers.TextVectorization(
    max_tokens=None,  #no maximum vocabulary
    output_sequence_length=MAX_QUERY_LENGTH,  #pad or truncate output to value
    output_mode="int",  #vector has index of vocabulary
    standardize="lower_and_strip_punctuation",  #convert input to lowercase and rmv punctuation
    split="whitespace",  #split values based on whitespace
    ngrams=1  #only look at whole words
)
vectorize_slot_text = keras.layers.TextVectorization(
    max_tokens=None,  #no maximum vocabulary
    output_sequence_length=MAX_QUERY_LENGTH,
    output_mode="int",  #vector has index of vocabulary
    standardize="lower",  #convert input to lowercase [can't do punctuation b/c of dashes]
    split="whitespace",  #split values based on whitespace
    ngrams=1  #only look at whole words
)

# CREATE VOCABULARY AND VECTORIZED TRAINING DATA
vectorize_query_text.adapt(train_query)  #build vocabulary
query_train = vectorize_query_text(train_query)  #vectorized training queries

vectorize_slot_text.adapt(train_slotfilling)  #build slot vocabulary
slots_train = vectorize_slot_text(train_slotfilling)  #vectorized training slots

#### BEGIN STREAMLIT APPLICATION ####
st.title("Day-Trader GPT")
st.write()
st.write("Enter your question into the following textbox:")

prompt = st.text_area(label="Prompt:",
                      value="")
run_query_button = st.button("Run query",
                             type="primary")

if run_query_button:
    formatted_prompt = re.sub(r'[0-9]', '', prompt)
    slot_filling = predict_slots_query(formatted_prompt,
                                    transformer_model,
                                    vectorize_query_text,
                                    vectorize_slot_text)

    return_df, written_query, all_data = SlotParser(slot_filling,
                                                    prompt,
                                                    stock_data)
    return_df.columns = [" ".join([x.capitalize() for x in col.split("_")]) for col in return_df.columns]
    st.data_editor(return_df,
                hide_index=True,
                key="filtered_df",
                use_container_width=True,
                disabled=True)

    st.write("Prompt you submitted:")
    st.info(prompt)

    st.write("Slot filling procedure (removes number requests):")
    st.info(slot_filling)

    st.write("SQL Query:")
    st.info(written_query)

    st.write("Entire Table:")
    st.data_editor(all_data,
                hide_index=True,
                use_container_width=True,
                disabled=True)